In [80]:
import os
import pandas as pd
import numpy as np

In [81]:
WINDOW_SIZE = 25
NUMBER_OF_FEATURES = 4
NUMBER_OF_CLASSES = 2

MOTION_CLASSES = ['STAND', 'STEPS']
MOTIONTYPE_CLASSES = ['SML', 'MED', 'LAR']
MOTIONSPEED_CLASSES = ['SLOW', 'AVERAGE', 'FAST']


# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)

DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\TRAIN2\\'

# numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TRAIN_CLASSIFY_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-X-TRAIN-CLASSIFY-V0.npy'
ALL_Y_TRAIN_CLASSIFY_MOTION_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTION-V0.npy'
ALL_Y_TRAIN_CLASSIFY_MOTIONTYPE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONTYPE-V0.npy'
ALL_Y_TRAIN_CLASSIFY_MOTIONSPEED_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\training\\ALL-Y-TRAIN-CLASSIFY-MOTIONSPEED-V0.npy'

In [82]:
LABEL_TO_CATEGORY_MOTION = {label: category for category, label in enumerate(MOTION_CLASSES)}
LABEL_TO_CATEGORY_MOTIONTYPE = {label: category for category, label in enumerate(MOTIONTYPE_CLASSES)}
LABEL_TO_CATEGORY_MOTIONSPEED = {label: category for category, label in enumerate(MOTIONSPEED_CLASSES)}

In [83]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TRAIN2-STAND1.xlsx', 'PROC-TRAIN2-STAND2.xlsx', 'PROC-TRAIN2-STAND3.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-100BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-110BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-200BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-50BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-55BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-60BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-LAR-80BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-100BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-110BPM.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-120BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-160BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-200BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-220BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-30BPM-AUGMENT.xlsx', 'PROC-TRAIN2-STEPS-LR-MED-40BPM-AUGMENT.xlsx', 'PROC-TRAIN2-S

In [84]:
def normalizeSensorData(sensor_input):
    return (sensor_input + 180)/360


In [85]:
def df2Xy(df, windowSize=5):
  X = []
  y_Motion= []
  y_MotionType = []
  y_MotionSpeed = []

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values = df.loc[i + j, ['L_Pitch', 'L_Roll', 'R_Pitch', 'R_Roll']].values.tolist()

        # add row values to the input
        input.append(row_values)

    # add our input to our total inputs, marked as X
    X.append(input)

  y_Motion = df.iloc[windowSize - 1:, df.columns.get_loc('Class_Motion')].tolist()
  y_MotionType = df.iloc[windowSize - 1:, df.columns.get_loc('Class_MotionType')].tolist()
  y_MotionSpeed = df.iloc[windowSize - 1:, df.columns.get_loc('Class_MotionSpeed')].tolist()


  return (np.array(X), np.array(y_Motion), np.array(y_MotionType), np.array(y_MotionSpeed))

In [86]:
ALL_X_TRAIN = np.empty((0, WINDOW_SIZE, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_TRAIN_MOTION = np.empty((0))  # List to store all Y training data
ALL_Y_TRAIN_MOTIONTYPE = np.empty((0))  # List to store all Y training data
ALL_Y_TRAIN_MOTIONSPEED = np.empty((0))  # List to store all Y training data

In [87]:
def processData(ALL_X_TRAIN, ALL_Y_TRAIN_MOTION, ALL_Y_TRAIN_MOTIONTYPE, ALL_Y_TRAIN_MOTIONSPEED):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        # NORMALIZE THE SENSOR DATA
        df["L_Pitch"] = df["L_Pitch"].apply(normalizeSensorData)
        df["L_Roll"] = df["L_Roll"].apply(normalizeSensorData)
        df["R_Pitch"] = df["R_Pitch"].apply(normalizeSensorData)
        df["R_Roll"] = df["R_Roll"].apply(normalizeSensorData)

        xTrain, yTrain_Motion, yTrain_MotionType, yTrain_MotionSpeed = df2Xy(df, WINDOW_SIZE)
        
        ALL_X_TRAIN = np.concatenate([ALL_X_TRAIN, xTrain])
        ALL_Y_TRAIN_MOTION = np.concatenate([ALL_Y_TRAIN_MOTION, yTrain_Motion])
        ALL_Y_TRAIN_MOTIONTYPE = np.concatenate([ALL_Y_TRAIN_MOTIONTYPE, yTrain_MotionType])
        ALL_Y_TRAIN_MOTIONSPEED = np.concatenate([ALL_Y_TRAIN_MOTIONSPEED, yTrain_MotionSpeed])
        
        print('xTrain.shape:', xTrain.shape)
        print('ALL_X_TRAIN.shape:', ALL_X_TRAIN.shape, ' ALL_Y_TRAIN_CURRVEL.shape:', ALL_Y_TRAIN_MOTION.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_motion_label = np.vectorize(LABEL_TO_CATEGORY_MOTION.get)(ALL_Y_TRAIN_MOTION)
    numerical_motiontype_label = np.vectorize(LABEL_TO_CATEGORY_MOTIONTYPE.get)(ALL_Y_TRAIN_MOTIONTYPE)
    numerical_motionspeed_label = np.vectorize(LABEL_TO_CATEGORY_MOTIONSPEED.get)(ALL_Y_TRAIN_MOTIONSPEED)
    

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TRAIN_CLASSIFY_PATH, ALL_X_TRAIN)
    np.save(ALL_Y_TRAIN_CLASSIFY_MOTION_PATH, numerical_motion_label)
    np.save(ALL_Y_TRAIN_CLASSIFY_MOTIONTYPE_PATH, numerical_motiontype_label)
    np.save(ALL_Y_TRAIN_CLASSIFY_MOTIONSPEED_PATH, numerical_motionspeed_label)


In [88]:
if not os.path.exists(ALL_X_TRAIN_CLASSIFY_PATH):
    processData(ALL_X_TRAIN, ALL_Y_TRAIN_MOTION, ALL_Y_TRAIN_MOTIONTYPE, ALL_Y_TRAIN_MOTIONSPEED)

ALL_X_TRAIN_CLASSIFY = np.load(ALL_X_TRAIN_CLASSIFY_PATH)
ALL_Y_TRAIN_CLASSIFY_MOTION = np.load(ALL_Y_TRAIN_CLASSIFY_MOTION_PATH)
ALL_Y_TRAIN_CLASSIFY_MOTIONTYPE = np.load(ALL_Y_TRAIN_CLASSIFY_MOTIONTYPE_PATH)
ALL_Y_TRAIN_CLASSIFY_MOTIONSPEED = np.load(ALL_Y_TRAIN_CLASSIFY_MOTIONSPEED_PATH)

print(ALL_X_TRAIN_CLASSIFY.shape)
print(ALL_Y_TRAIN_CLASSIFY_MOTION.shape)


xTrain.shape: (1335, 25, 4)
ALL_X_TRAIN.shape: (1335, 25, 4)  ALL_Y_TRAIN_CURRVEL.shape: (1335,)
-------------------------------
xTrain.shape: (3288, 25, 4)
ALL_X_TRAIN.shape: (4623, 25, 4)  ALL_Y_TRAIN_CURRVEL.shape: (4623,)
-------------------------------
xTrain.shape: (1743, 25, 4)
ALL_X_TRAIN.shape: (6366, 25, 4)  ALL_Y_TRAIN_CURRVEL.shape: (6366,)
-------------------------------
xTrain.shape: (3381, 25, 4)
ALL_X_TRAIN.shape: (9747, 25, 4)  ALL_Y_TRAIN_CURRVEL.shape: (9747,)
-------------------------------
xTrain.shape: (3274, 25, 4)
ALL_X_TRAIN.shape: (13021, 25, 4)  ALL_Y_TRAIN_CURRVEL.shape: (13021,)
-------------------------------
xTrain.shape: (3773, 25, 4)
ALL_X_TRAIN.shape: (16794, 25, 4)  ALL_Y_TRAIN_CURRVEL.shape: (16794,)
-------------------------------
xTrain.shape: (3072, 25, 4)
ALL_X_TRAIN.shape: (19866, 25, 4)  ALL_Y_TRAIN_CURRVEL.shape: (19866,)
-------------------------------
xTrain.shape: (3381, 25, 4)
ALL_X_TRAIN.shape: (23247, 25, 4)  ALL_Y_TRAIN_CURRVEL.shape: (